In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
%cd '/content/drive/My Drive/BTP_MUTLICLASS_YOLO/'

In [ ]:
# !git clone --q https://github.com/AlexeyAB/darknet/

In [ ]:
%cd '/content/drive/My Drive/BTP_MUTLICLASS_YOLO/darknet/'
# # YOLOV4 pretrained weights

!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137

In [ ]:
# !wget https://pjreddie.com/media/files/darknet53.conv.74

In [ ]:
%cd '/content/drive/My Drive/BTP_MUTLICLASS_YOLO/darknet/'  
!make
!chmod +x '/content/drive/My Drive/BTP_MUTLICLASS_YOLO/darknet/' 

In [ ]:
!sudo apt install dos2unix

In [ ]:
# %cp -r '/content/drive/My Drive/Data/traffic/' '/content/drive/My Drive/Data/Sun_hat/darknet/data/'

In [ ]:
# import os
# path_to_images = '/content/drive/My Drive/BTP_MUTLICLASS_YOLO/train/'   
# file_path = '/content/drive/My Drive/BTP_MUTLICLASS_YOLO/darknet/data/train.txt'
# f = open(file_path,'w+')
# count=0
# for each in os.listdir(path_to_images):
#   if each.endswith('.jpg'):
#     # print(os.path.join(path_to_images,each))
#     count+=1
#     f.write(os.path.join(path_to_images,each)+'\n')
# print(count)

In [ ]:
# import os
# path_to_images = '/content/drive/My Drive/BTP_MUTLICLASS_YOLO/test/'   
# file_path = '/content/drive/My Drive/BTP_MUTLICLASS_YOLO/darknet/data/test.txt'
# f = open(file_path,'w+')
# count=0
# for each in os.listdir(path_to_images):
#   if each.endswith('.jpg'):
#     print(os.path.join(path_to_images,each))
#     count+=1
#     f.write(os.path.join(path_to_images,each)+'\n')
# print(count)

In [ ]:
!dos2unix '/content/drive/My Drive/BTP_MUTLICLASS_YOLO/darknet/data/train.txt'
!dos2unix '/content/drive/My Drive/BTP_MUTLICLASS_YOLO/darknet/data/test.txt'
!dos2unix '/content/drive/My Drive/BTP_MUTLICLASS_YOLO/darknet/data/yolo.data'
!dos2unix '/content/drive/My Drive/BTP_MUTLICLASS_YOLO/darknet/data/yolo.names'
!dos2unix '/content/drive/My Drive/BTP_MUTLICLASS_YOLO/darknet/yolov4-custom.cfg'

In [ ]:
 # %cd '/content/drive/My Drive/Data/Yolo/darknet/'

In [ ]:
# If your output cell shows something similar like belows, then congratulation, your model has started training successfully!
%cd '/content/drive/My Drive/BTP_MUTLICLASS_YOLO/darknet/'
# !./darknet detector train data/yolo.data yolov3-spp-custom.cfg darknet53.conv.74 -map -dont_show
!./darknet detector train data/yolo.data yolov4-custom.cfg  yolov4.conv.137 -map -dont_show

In [ ]:
# %cd '/content/drive/My Drive/Data/helmet/darknet/'

In [ ]:
#!./darknet detector map data/yolo.data yolov3-spp-custom.cfg backup/yolov3-spp-custom_best.weights test/BikesHelmets415_png.rf.12ea09fac636020a3b9ce679cd6453d5.jpg

In [ ]:
# net = cv2.dnn.readNet("/content/drive/My Drive/Data/Yolo/darknet/backup/yolov3-spp-custom_last.weights", "/content/drive/My Drive/Data/Yolo/darknet/yolov3_testing.cfg")
# # Name custom object
# classes = ["hairband"]
# # Images path
# images_path = glob.glob(r"D:\Pysource\Youtube205) Train Yolo google cloud\dataset\*.jpg")

In [ ]:
import cv2
import numpy as np
import glob
import random
import matplotlib.pyplot as plt

# Load Yolo
net = cv2.dnn.readNetFromDarknet("/content/drive/My Drive/Data/helmet/darknet/backup/yolov4-custom_best.weights","/content/drive/My Drive/Data/helmet/darknet/yolov4-custom.cfg")

# Name custom object
classes = ["With helmet","Without helmet"]

# Images path
images_path = glob.glob("/content/drive/My Drive/Data/helmet/test/BikesHelmets415_png.rf.12ea09fac636020a3b9ce679cd6453d5.jpg")



layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
colors = np.random.uniform(0, 255, size=(len(classes), 3))

# Insert here the path of your images
random.shuffle(images_path)
# loop through all the images
for img_path in images_path:
    # Loading image
    # print(img_path)
    img = cv2.imread(img_path)
    img = cv2.resize(img, None, fx=0.4, fy=0.4)
    height, width, channels = img.shape

    # Detecting objects
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)

    net.setInput(blob)
    outs = net.forward(output_layers)

    # Showing informations on the screen
    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.1:
                # Object detected
                print(class_id)
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.1, 0.1)
    print(indexes)
    font = cv2.FONT_HERSHEY_PLAIN
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            color = colors[class_ids[i]]
            cv2.rectangle(img, (x, y), (x + w, y + h), color, 1)
            cv2.putText(img, label, (x, y + 30), font, 1, color, 1)

    plt.imshow(img)
    # cv2.imshow("Image", img)
    # key = cv2.waitKey(0)

# cv2.destroyAllWindows()

In [ ]:
!./darknet detector map data/yolo.data yolov4-custom.cfg backup/yolov4-custom_best.weights



In [ ]:
!./darknet detector test data/yolo.data yolov4-custom.cfg backup/yolov4-custom_best.weights test/BikesHelmets23_png.rf.97fa48d8900b75c40cd7072373030ac5.jpg